In [121]:
# @title Import Necessary Packages
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from random import random
from random import randint

In [122]:
# @title Mount Google Drive
from google.colab import drive # Load data from google drive
drive.mount("/content/drive")

%cd "/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/data"
%ls -f

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/data
1_cleaned_bonds_data.csv      7_credit_spread.csv
2_final_company_info.csv      preprocess_credit_spread.ipynb
5_matched_stock_prices/       8_marco_econ_metrics_month.csv
8_income_stmt/                preprocess_financials.ipynb
6_bond_prices/                5_financials.csv
9_balance_sheet/              1_cleaned_bonds_data.gsheet
7_cash_flow/                  3_Issue_Ratings.csv
company_financial_ratios/     preprocess_ratings.ipynb
6_marco_econ_metrics.csv      4_Issuer_Ratings.csv
Credit_spread.csv             3_Issue_Ratings.gsheet
cleaned_stock_price/          4_Issuer_Ratings.gsheet
9_stock_price_month.csv       9_stock_price_month.gsheet
preprocess_stock_price.ipynb


In [123]:
# @title Read CSV files
bonds = pd.read_csv("1_cleaned_bonds_data.csv")
company_info = pd.read_csv("2_final_company_info.csv")
Bonds_Ratings = pd.read_csv("3_Issue_Ratings.csv")
Issuer_Ratings = pd.read_csv("4_Issuer_Ratings.csv")
financials = pd.read_csv("5_financials.csv")
econ_metrics = pd.read_csv('8_marco_econ_metrics_month.csv')

bonds['RIC'].unique().shape # RIC as the key Identifier

(1503,)

In [124]:
Issuer_Ratings.info()
Bonds_Ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88078 entries, 0 to 88077
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PermID      88078 non-null  int64  
 1   RatingRank  88078 non-null  float64
 2   Month       88078 non-null  int64  
 3   Year        88078 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 2.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43097 entries, 0 to 43096
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISIN         43097 non-null  object 
 1   Rating Rank  43097 non-null  float64
 2   Month        43097 non-null  int64  
 3   Year         43097 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ MB


In [125]:
# @title Merge Bond_Info, Company_Info, Issuer_Ratings, and Bond_Rating

# Merge Bonds_info and Company_info
bonds_company = bonds.merge(company_info, left_on='PermID', right_on='OAPermID')
bonds_company = bonds_company.drop('OAPermID', axis = 1)

# Merge Bonds_info, Company_info, and Issuer Ratings
bonds_company_IssuerR = bonds_company.merge(Issuer_Ratings, on = ['PermID'], how = 'left')
bonds_company_IssuerR = bonds_company_IssuerR.rename(columns = {"RatingRank": "Issuer Rating Rank"})

# Merge Bonds_info, Comapny_info, Issuer Ratings, and Bonds Ratings
bonds_company_IssuerR_BondsR = bonds_company_IssuerR.merge(Bonds_Ratings, on = ['ISIN', 'Year', 'Month'],)
bonds_company_IssuerR_BondsR = bonds_company_IssuerR_BondsR.rename(columns={'Rating Rank': 'Bond Rating Rank'})
bonds_company_IssuerR_BondsR = bonds_company_IssuerR_BondsR.drop(['IssuerOAPermID', "PrimaryRIC", "PrimaryRICExchangeCode", "PrimaryRICTickerSymbol"], axis = 1) # drop unnecessary identifiers

combined = bonds_company_IssuerR_BondsR.reset_index(drop = True)
combined = combined[(combined["Bond Rating Rank"] <= 12)]

In [126]:
print(bonds_company.shape, bonds_company['RIC'].unique().shape[0])
print(bonds_company_IssuerR.shape, bonds_company_IssuerR['RIC'].unique().shape[0])
print(bonds_company_IssuerR_BondsR.shape, bonds_company_IssuerR_BondsR['RIC'].unique().shape[0])
# combined.info()
# combined.head(20)

(1503, 26) 1503
(182704, 29) 1503
(41051, 26) 1032


In [127]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39902 entries, 0 to 41050
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BondID                  39902 non-null  float64
 1   PermID                  39902 non-null  int64  
 2   ISIN                    39902 non-null  object 
 3   IssuePrice              39643 non-null  float64
 4   IssueDate               39902 non-null  object 
 5   AssetStatusDescription  39902 non-null  object 
 6   CouponFrequency         39902 non-null  float64
 7   CouponRate              39902 non-null  float64
 8   DenominationMinimum     39902 non-null  float64
 9   FaceIssuedTotal         39902 non-null  float64
 10  FaceIssuedUSD           39902 non-null  int64  
 11  FaceOutstanding         39902 non-null  float64
 12  FaceOutstandingDate     39902 non-null  object 
 13  FaceOutstandingUSD      39902 non-null  int64  
 14  MoodysRatingDate        39902 non-null

In [128]:
combined = combined.drop_duplicates()
print(combined.duplicated().any())
print(combined.shape)

False
(39396, 26)


In [129]:
# @title Put Identifier in the first Column

columns_to_convert = ['BondID', 'RIC'] # Retype Identifiers to String
combined[columns_to_convert] = combined[columns_to_convert].astype(object)

combined['PermID'] = combined['PermID'].astype(int)
combined['Bond Rating Rank'] = np.rint(combined['Bond Rating Rank'])

RIC = combined.pop('RIC').rename("RIC").astype(object)
combined = pd.concat([RIC, combined], axis = 1) # Move Key Identifier to the first Column

<ipython-input-129-cc0fb062effb>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-129-cc0fb062effb>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-129-cc0fb062effb>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [130]:
print(combined['RIC'].unique().shape[0])

1020


In [131]:
data = combined
y = pd.concat([data['RIC'], data['Bond Rating Rank']], axis = 1)
x = data

In [132]:
# @title Generate Y-value
x_grouped = x.iloc[:, :].groupby('RIC')
y_grouped = y.iloc[:, :].groupby('RIC')

new_x = pd.DataFrame()
for name, x_group in x_grouped: # drop the last one as we do not know its prediciton
  # print(x_group.shape[0], end = " ")
  x_group = x_group.iloc[:-1, :]
  # print(x_group.shape[0], end = " ")
  new_x = pd.concat([new_x, x_group], axis = 0)
  # print()
  # print(name, ": ", x_group['Bond Rating Rank'])

print()
new_y = pd.DataFrame([])
for name, y_group in y_grouped: # drop the first one as there is nothing to predict
  # print(y_group.shape[0], end = " ")
  y_group = y_group.iloc[1:, :]
  # print(y_group.shape[0], end = " ")
  new_y = pd.concat([new_y, y_group], axis = 0)
  # print()
  # print(name, ": ", y_group['Bond Rating Rank'])

print("Number of Data of x: ", new_x.shape[0])
print("Number of Data of y: ", new_y.shape[0])


Number of Data of x:  38376
Number of Data of y:  38376


In [133]:
new_x = new_x.reset_index(drop=True)
new_y = new_y.rename(columns={'Bond Rating Rank': 'Y'}).reset_index(drop=True)
data = pd.concat([new_x, np.rint(new_y['Y'])], axis = 1)

In [134]:
print(new_x['RIC'].unique().shape[0])

844


In [135]:
for name, group in data.groupby("Year"):
  print(name, group.shape)

2011.0 (20, 27)
2012.0 (186, 27)
2013.0 (343, 27)
2014.0 (694, 27)
2015.0 (1115, 27)
2016.0 (1690, 27)
2017.0 (2238, 27)
2018.0 (2832, 27)
2019.0 (3839, 27)
2020.0 (5086, 27)
2021.0 (6783, 27)
2022.0 (8579, 27)
2023.0 (4971, 27)


In [136]:
print(data['RIC'].unique().shape)

(844,)


In [137]:
data.info()
data.iloc[:, -5:].head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38376 entries, 0 to 38375
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   RIC                     38376 non-null  object 
 1   BondID                  38376 non-null  object 
 2   PermID                  38376 non-null  int64  
 3   ISIN                    38376 non-null  object 
 4   IssuePrice              38125 non-null  float64
 5   IssueDate               38376 non-null  object 
 6   AssetStatusDescription  38376 non-null  object 
 7   CouponFrequency         38376 non-null  float64
 8   CouponRate              38376 non-null  float64
 9   DenominationMinimum     38376 non-null  float64
 10  FaceIssuedTotal         38376 non-null  float64
 11  FaceIssuedUSD           38376 non-null  int64  
 12  FaceOutstanding         38376 non-null  float64
 13  FaceOutstandingDate     38376 non-null  object 
 14  FaceOutstandingUSD      38376 non-null

,Issuer Rating Rank,Month,Year,Bond Rating Rank,Y
0,8.333333,11.0,2018.0,10.0,10.0
1,8.333333,12.0,2018.0,10.0,10.0
2,8.333333,1.0,2019.0,10.0,10.0
3,8.333333,2.0,2019.0,10.0,10.0
4,8.333333,3.0,2019.0,10.0,9.0
5,8.333333,4.0,2019.0,9.0,9.0
6,8.333333,5.0,2019.0,9.0,9.0
7,8.333333,6.0,2019.0,9.0,9.0
8,8.333333,7.0,2019.0,9.0,9.0
9,8.333333,8.0,2019.0,9.0,9.0


In [138]:
# @title Handle Finacials.file

financials = financials.drop("Dividend Growth", axis = 1)

financials['Date'] = (pd.to_datetime(financials['Date'], format = "%Y-%m-%d"))
financials['Year'] = financials['Date'].dt.year
financials.drop(['Date'], axis=1, inplace = True)

financials['PermID'] = financials['PermID'].astype(int)

In [139]:
financials.info()
financials.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 0 to 896
Data columns (total 29 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   PermID                                             897 non-null    int64  
 1   Tier 1 Capital Ratio                               897 non-null    float64
 2   Total Capital Ratio                                897 non-null    float64
 3   Non Performing Loan Ratio                          897 non-null    float64
 4   Coverage Ratio                                     897 non-null    float64
 5   Equity at Risk                                     897 non-null    float64
 6   Efficiency Ratio                                   897 non-null    float64
 7   Loan to Deposit Ratio                              897 non-null    float64
 8   CASA Ratio                                         897 non-null    float64
 9   CD Non Equ

,PermID,Tier 1 Capital Ratio,Total Capital Ratio,Non Performing Loan Ratio,Coverage Ratio,Equity at Risk,Efficiency Ratio,Loan to Deposit Ratio,CASA Ratio,CD Non Equity Ratio,...,Total Deposits,Total Debt,Cash and Equivalents,Total Equity,Net Income After Taxes,Net Interest Margin,ROAA,ROAE,Liquidity Coverage Ratio,Year
0,4295856152,10.010000,11.700000,0.010441,0.971682,0.284121,0.519835,1.199045,0.920218,0.660782,...,6.152028e+11,1.936289e+11,1.953238e+10,5.422481e+10,9.455672e+09,-0.133292,0.009597,0.174379,6.422221,2011
1,4295856152,10.000000,11.000000,0.008804,1.029870,0.231817,0.502522,1.143331,0.921193,0.678325,...,6.782431e+11,2.002198e+11,4.506859e+10,6.054138e+10,1.037321e+10,-0.121495,0.009782,0.171341,5.444048,2012
2,4295856152,10.300000,11.200000,0.026987,0.293343,0.436489,0.508647,1.160218,0.923144,0.676835,...,7.077430e+11,2.210046e+11,2.271279e+10,6.638147e+10,1.126125e+10,-0.056387,0.010127,0.169644,6.422221,2013
3,4295856152,11.100000,12.000000,0.026945,0.248011,0.398558,0.498797,1.116548,0.927045,0.691276,...,7.572931e+11,2.417761e+11,6.658062e+10,7.200325e+10,1.275999e+10,-0.026004,0.010929,0.177214,7.400394,2014
4,4295856152,11.200000,12.700000,0.025460,0.229016,0.370315,0.498246,1.080912,0.904399,0.693605,...,8.400354e+11,2.707789e+11,7.740375e+10,7.734327e+10,1.338546e+10,-0.016080,0.010389,0.173066,6.911307,2015
5,4295856152,11.250000,13.050000,0.024718,0.219519,0.356194,0.425856,1.101690,0.879350,0.693466,...,8.930329e+11,2.926007e+11,7.691842e+10,8.852928e+10,1.322466e+10,-0.000648,0.009609,0.149382,6.911307,2016
6,4295856152,11.775000,14.025000,0.022913,0.228166,0.317966,0.419804,1.092570,0.874112,0.707431,...,9.529857e+11,3.057885e+11,7.494173e+10,9.310223e+10,1.397109e+10,0.011053,0.009701,0.150062,5.652314,2017
7,4295856152,12.300000,15.000000,0.021109,0.236813,0.279737,0.443506,1.125565,0.874984,0.701167,...,9.390190e+11,3.117156e+11,7.832719e+10,9.928603e+10,1.337955e+10,0.017570,0.009301,0.134758,5.296880,2018
8,4295856152,12.670000,15.480000,0.021769,0.295556,0.296980,0.468884,1.124471,0.869750,0.712893,...,9.537218e+11,3.061292e+11,7.801593e+10,1.026612e+11,1.234990e+10,0.011895,0.008573,0.120298,4.165923,2019
9,4295856152,12.485000,15.240000,0.021439,0.266184,0.288358,0.463538,1.058286,0.868945,0.743564,...,1.034933e+12,2.819988e+11,9.789202e+10,1.061115e+11,1.089836e+10,0.048680,0.007275,0.102707,6.427838,2020


In [140]:
# @title Merge Financials

# common = np.intersect1d(financials['PermID'].unique(), data['PermID'].unique())
# data_financials = data[data['PermID'].astype(int).isin(common)]

data_financials = data.merge(financials, on=['PermID', 'Year'])
data_financials = data_financials.sort_values(by = ['RIC', 'Year']).reset_index(drop = True)

data = data_financials

In [141]:
print(data_financials['RIC'].unique().shape)
data_financials.info()
data_financials.head(5)

(478,)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23510 entries, 0 to 23509
Data columns (total 54 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   RIC                                                23510 non-null  object 
 1   BondID                                             23510 non-null  object 
 2   PermID                                             23510 non-null  int64  
 3   ISIN                                               23510 non-null  object 
 4   IssuePrice                                         23259 non-null  float64
 5   IssueDate                                          23510 non-null  object 
 6   AssetStatusDescription                             23510 non-null  object 
 7   CouponFrequency                                    23510 non-null  float64
 8   CouponRate                                         23510 non-null  float64
 9  

,RIC,BondID,PermID,ISIN,IssuePrice,IssueDate,AssetStatusDescription,CouponFrequency,CouponRate,DenominationMinimum,...,"Total Assets, Reported",Total Deposits,Total Debt,Cash and Equivalents,Total Equity,Net Income After Taxes,Net Interest Margin,ROAA,ROAE,Liquidity Coverage Ratio
0,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,82257368799,6.146387e+10,6.392883e+09,7.152229e+09,1.245327e+10,981308411.2,0.103260,0.011930,0.078799,38.081340
1,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,82257368799,6.146387e+10,6.392883e+09,7.152229e+09,1.245327e+10,981308411.2,0.103260,0.011930,0.078799,38.081340
2,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,88571171819,6.526420e+10,8.053559e+09,1.209561e+10,1.234274e+10,327102803.7,0.095796,0.003693,0.026502,2.469725
3,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,88571171819,6.526420e+10,8.053559e+09,1.209561e+10,1.234274e+10,327102803.7,0.095796,0.003693,0.026502,2.469725
4,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,88571171819,6.526420e+10,8.053559e+09,1.209561e+10,1.234274e+10,327102803.7,0.095796,0.003693,0.026502,2.469725


In [142]:
data_financials_csv = data_financials.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/data_financials.csv', 'w') as f:
  f.write(data_financials_csv)

In [143]:
econ_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3289 entries, 0 to 3288
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    3289 non-null   int64  
 1   Country       3289 non-null   object 
 2   FullName      3289 non-null   object 
 3   Date          3289 non-null   object 
 4   M3            3289 non-null   float64
 5   RealGDP       3289 non-null   float64
 6   PPI           3289 non-null   float64
 7   Unemployment  3289 non-null   float64
 8   InterestRate  3289 non-null   float64
 9   CoreCPI       3289 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 257.1+ KB


In [144]:
# @title Merge Economic Metrics
econ_metrics = econ_metrics.drop(["Unnamed: 0", "FullName"], axis = 1)
econ_metrics['Date'] = pd.to_datetime(econ_metrics['Date'], format = "%d/%m/%Y")
econ_metrics['Month'] = econ_metrics['Date'].dt.month
econ_metrics['Year'] = econ_metrics['Date'].dt.year
econ_metrics = econ_metrics.drop("Date", axis = 1)

data_econ = data.merge(econ_metrics, left_on=['Year', 'Month', 'CountryHeadquarters',], right_on=['Year', 'Month','Country', ])
data_econ = data_econ.sort_values(by = ['RIC', 'Year']).reset_index(drop = True)

data = data_econ

In [145]:
data_econ_csv = data_econ.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/data_econ.csv', 'w') as f:
  f.write(data_econ_csv)

In [146]:
print(data_econ['RIC'].unique().shape)

(478,)


In [147]:
total_num = 0
for name, group in data_econ.groupby("Year"):
  print(name, group['RIC'].unique().shape)
  total_num += group['RIC'].unique().shape[0]

2011.0 (5,)
2012.0 (17,)
2013.0 (29,)
2014.0 (46,)
2015.0 (66,)
2016.0 (101,)
2017.0 (138,)
2018.0 (180,)
2019.0 (232,)
2020.0 (282,)
2021.0 (356,)
2022.0 (446,)
2023.0 (459,)


In [148]:
for name, group in data.groupby(by = ["Year", "Month"]):
    print(name, group.shape, group['RIC'].unique().shape)

(2011.0, 9.0) (5, 61) (5,)
(2011.0, 10.0) (5, 61) (5,)
(2011.0, 11.0) (5, 61) (5,)
(2011.0, 12.0) (5, 61) (5,)
(2012.0, 1.0) (6, 61) (6,)
(2012.0, 2.0) (6, 61) (6,)
(2012.0, 3.0) (6, 61) (6,)
(2012.0, 4.0) (6, 61) (6,)
(2012.0, 5.0) (10, 61) (10,)
(2012.0, 6.0) (15, 61) (15,)
(2012.0, 7.0) (16, 61) (16,)
(2012.0, 8.0) (16, 61) (16,)
(2012.0, 9.0) (16, 61) (16,)
(2012.0, 10.0) (16, 61) (16,)
(2012.0, 11.0) (16, 61) (16,)
(2012.0, 12.0) (17, 61) (17,)
(2013.0, 1.0) (18, 61) (18,)
(2013.0, 2.0) (19, 61) (19,)
(2013.0, 3.0) (19, 61) (19,)
(2013.0, 4.0) (20, 61) (20,)
(2013.0, 5.0) (20, 61) (20,)
(2013.0, 6.0) (20, 61) (20,)
(2013.0, 7.0) (23, 61) (23,)
(2013.0, 8.0) (23, 61) (23,)
(2013.0, 9.0) (23, 61) (23,)
(2013.0, 10.0) (26, 61) (26,)
(2013.0, 11.0) (29, 61) (29,)
(2013.0, 12.0) (29, 61) (29,)
(2014.0, 1.0) (30, 61) (30,)
(2014.0, 2.0) (34, 61) (34,)
(2014.0, 3.0) (35, 61) (35,)
(2014.0, 4.0) (36, 61) (36,)
(2014.0, 5.0) (37, 61) (37,)
(2014.0, 6.0) (39, 61) (39,)
(2014.0, 7.0) (39, 61

In [149]:
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23000 entries, 0 to 22999
Data columns (total 61 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   RIC                                                23000 non-null  object 
 1   BondID                                             23000 non-null  object 
 2   PermID                                             23000 non-null  int64  
 3   ISIN                                               23000 non-null  object 
 4   IssuePrice                                         22753 non-null  float64
 5   IssueDate                                          23000 non-null  object 
 6   AssetStatusDescription                             23000 non-null  object 
 7   CouponFrequency                                    23000 non-null  float64
 8   CouponRate                                         23000 non-null  float64
 9   Denomi

,RIC,BondID,PermID,ISIN,IssuePrice,IssueDate,AssetStatusDescription,CouponFrequency,CouponRate,DenominationMinimum,...,ROAA,ROAE,Liquidity Coverage Ratio,Country,M3,RealGDP,PPI,Unemployment,InterestRate,CoreCPI
0,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.011930,0.078799,38.081340,IRL,6.5,7.2,3.15,5.3,0.0,0.2
1,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.011930,0.078799,38.081340,IRL,10.7,3.5,-2.26,5.3,0.0,0.5
2,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,11.0,3.5,0.10,5.7,0.0,0.9
3,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,7.1,3.5,0.10,5.6,0.0,0.7
4,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,5.3,4.3,0.42,5.4,0.0,1.0
5,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,2.9,4.3,0.94,4.6,0.0,1.4
6,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,6.3,4.3,-0.72,4.5,0.0,0.5
7,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,4.7,4.3,-2.42,4.5,0.0,1.0
8,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,8.7,4.3,-2.04,5.3,0.0,0.5
9,00135TAA2=,192814000000.0,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,0.003693,0.026502,2.469725,IRL,4.4,4.3,-1.23,5.3,0.0,0.7


In [150]:
# @title Make it to 3-Years Format

# grouped = data.groupby('RIC')

# data_3_years = pd.DataFrame()
# for name, group in grouped:
#   drop_num = group.shape[0] % 3
#   group = group[drop_num:]
#   data_3_years = pd.concat([data_3_years, group], axis = 0)

# data = data_3_years.reset_index(drop = True)

In [151]:
for name, group in data.groupby("Year"):
  print(name, group.shape)

2011.0 (20, 61)
2012.0 (146, 61)
2013.0 (269, 61)
2014.0 (467, 61)
2015.0 (700, 61)
2016.0 (1067, 61)
2017.0 (1478, 61)
2018.0 (1894, 61)
2019.0 (2527, 61)
2020.0 (3167, 61)
2021.0 (3885, 61)
2022.0 (4998, 61)
2023.0 (2382, 61)


In [152]:
# @title Format the data

identifiers = ['BondID', 'PermID',]
data[identifiers] = data[identifiers].astype(int).astype(object)
data = data.rename(columns = {'GDP': "Real GDP"})

Issuer_Rating_Rank = data.pop('Issuer Rating Rank')
Bond_Rating_Rank = data.pop('Bond Rating Rank')
Year = data.pop('Year')
Month = data.pop('Month')
Y = data.pop('Y')

data = pd.concat([data, Issuer_Rating_Rank, Bond_Rating_Rank, Year, Month, Y], axis = 1)

In [153]:
data.info()
data.head(31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23000 entries, 0 to 22999
Data columns (total 61 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   RIC                                                23000 non-null  object 
 1   BondID                                             23000 non-null  object 
 2   PermID                                             23000 non-null  object 
 3   ISIN                                               23000 non-null  object 
 4   IssuePrice                                         22753 non-null  float64
 5   IssueDate                                          23000 non-null  object 
 6   AssetStatusDescription                             23000 non-null  object 
 7   CouponFrequency                                    23000 non-null  float64
 8   CouponRate                                         23000 non-null  float64
 9   Denomi

,RIC,BondID,PermID,ISIN,IssuePrice,IssueDate,AssetStatusDescription,CouponFrequency,CouponRate,DenominationMinimum,...,RealGDP,PPI,Unemployment,InterestRate,CoreCPI,Issuer Rating Rank,Bond Rating Rank,Year,Month,Y
0,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,7.2,3.15,5.3,0.0,0.2,8.333333,10.0,2018.0,11.0,10.0
1,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,3.5,-2.26,5.3,0.0,0.5,8.333333,10.0,2018.0,12.0,10.0
2,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,3.5,0.10,5.7,0.0,0.9,8.333333,10.0,2019.0,1.0,10.0
3,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,3.5,0.10,5.6,0.0,0.7,8.333333,10.0,2019.0,2.0,10.0
4,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,0.42,5.4,0.0,1.0,8.333333,10.0,2019.0,3.0,9.0
5,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,0.94,4.6,0.0,1.4,8.333333,9.0,2019.0,4.0,9.0
6,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,-0.72,4.5,0.0,0.5,8.333333,9.0,2019.0,5.0,9.0
7,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,-2.42,4.5,0.0,1.0,8.333333,9.0,2019.0,6.0,9.0
8,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,-2.04,5.3,0.0,0.5,8.333333,9.0,2019.0,7.0,9.0
9,00135TAA2=,192814000000,5059082241,US00135TAA25,99.771,12/10/2018,Issued,2.0,4.75,200000.0,...,4.3,-1.23,5.3,0.0,0.7,8.333333,9.0,2019.0,8.0,9.0


In [154]:
# @title Drop Classes

# data_classes = data[data['Y'] <= 12]
# print(data['Y'].unique())

# grouped = data_classes.groupby('Y')
# for name, group in grouped:
#   print(name, group.shape)
# data = data_classes

In [155]:
# @title Create Exploratory Data Analysis
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def build_EDA_Plots(data):
  print(data.shape)
  fig = make_subplots(rows=data.shape[1] // 3 + 1, cols = 3, subplot_titles=data.columns.tolist())
  r = 1  # rowIndex
  c = 1  # colIndex

  for i, col in enumerate(data.columns):
      if type(data[col]) == object:
          trace = go.Bar(x=data[col].value_counts().index, y=data[col].value_counts().values)
      else:
          trace = go.Histogram(x=data[col])

      fig.add_trace(trace, row=r, col=c)
      fig.update_xaxes(title_text="Value", row=r, col=c)
      fig.update_yaxes(title_text="Count", row=r, col=c)

      c += 1
      if c > 3:
        c = 1
        r += 1

  fig.update_layout(title="EDA", height=4000, width=1300, showlegend=False)
  fig.show()

In [156]:
build_EDA_Plots(data)

Output hidden; open in https://colab.research.google.com to view.

In [157]:
# @title Handle Missing Values
print("Rows: ", data.shape[0])
print(f"Total Count of Missing Value:\n{data.isnull().sum()}")

data["IssuePrice"] = data["IssuePrice"].fillna(data["IssuePrice"].mean())
data['IsPublic'] = data['IsPublic'].fillna(False)
data["Ticker"] = data["Ticker"].fillna("BNS")

Rows:  23000
Total Count of Missing Value:
RIC                     0
BondID                  0
PermID                  0
ISIN                    0
IssuePrice            247
                     ... 
Issuer Rating Rank      0
Bond Rating Rank        0
Year                    0
Month                   0
Y                       0
Length: 61, dtype: int64


In [158]:
print("Rows: ", data.shape[0])
print(f"Total Count of Missing Value:\n{data.isnull().sum()}")

Rows:  23000
Total Count of Missing Value:
RIC                   0
BondID                0
PermID                0
ISIN                  0
IssuePrice            0
                     ..
Issuer Rating Rank    0
Bond Rating Rank      0
Year                  0
Month                 0
Y                     0
Length: 61, dtype: int64


In [159]:
for name, group in data.groupby(by = ["Year", "Month"]):
    print(name, group.shape, group['RIC'].unique().shape)

(2011.0, 9.0) (5, 61) (5,)
(2011.0, 10.0) (5, 61) (5,)
(2011.0, 11.0) (5, 61) (5,)
(2011.0, 12.0) (5, 61) (5,)
(2012.0, 1.0) (6, 61) (6,)
(2012.0, 2.0) (6, 61) (6,)
(2012.0, 3.0) (6, 61) (6,)
(2012.0, 4.0) (6, 61) (6,)
(2012.0, 5.0) (10, 61) (10,)
(2012.0, 6.0) (15, 61) (15,)
(2012.0, 7.0) (16, 61) (16,)
(2012.0, 8.0) (16, 61) (16,)
(2012.0, 9.0) (16, 61) (16,)
(2012.0, 10.0) (16, 61) (16,)
(2012.0, 11.0) (16, 61) (16,)
(2012.0, 12.0) (17, 61) (17,)
(2013.0, 1.0) (18, 61) (18,)
(2013.0, 2.0) (19, 61) (19,)
(2013.0, 3.0) (19, 61) (19,)
(2013.0, 4.0) (20, 61) (20,)
(2013.0, 5.0) (20, 61) (20,)
(2013.0, 6.0) (20, 61) (20,)
(2013.0, 7.0) (23, 61) (23,)
(2013.0, 8.0) (23, 61) (23,)
(2013.0, 9.0) (23, 61) (23,)
(2013.0, 10.0) (26, 61) (26,)
(2013.0, 11.0) (29, 61) (29,)
(2013.0, 12.0) (29, 61) (29,)
(2014.0, 1.0) (30, 61) (30,)
(2014.0, 2.0) (34, 61) (34,)
(2014.0, 3.0) (35, 61) (35,)
(2014.0, 4.0) (36, 61) (36,)
(2014.0, 5.0) (37, 61) (37,)
(2014.0, 6.0) (39, 61) (39,)
(2014.0, 7.0) (39, 61

In [160]:
# @title Output  Dataset
data_csv = data.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/Dataset_1.csv', 'w') as f:
  f.write(data_csv)

In [161]:
# @title Make Test Set of 2023
test_data = data.copy()
grouped = test_data.groupby('RIC')

test = pd.DataFrame()

for name, group in grouped:
  test = pd.concat([test, group[(group['Year'] > 2022)]], axis = 0)

data = data[data['Year'] < 2023]

In [162]:
print(test['RIC'].unique().shape)
print(test.shape)
print(test.columns)

(459,)
(2382, 61)
Index(['RIC', 'BondID', 'PermID', 'ISIN', 'IssuePrice', 'IssueDate',
       'AssetStatusDescription', 'CouponFrequency', 'CouponRate',
       'DenominationMinimum', 'FaceIssuedTotal', 'FaceIssuedUSD',
       'FaceOutstanding', 'FaceOutstandingDate', 'FaceOutstandingUSD',
       'MoodysRatingDate', 'Maturity', 'CommonName', 'CountryHeadquarters',
       'Ticker', 'IsPublic', 'BondsCount', 'Tier 1 Capital Ratio',
       'Total Capital Ratio', 'Non Performing Loan Ratio', 'Coverage Ratio',
       'Equity at Risk', 'Efficiency Ratio', 'Loan to Deposit Ratio',
       'CASA Ratio', 'CD Non Equity Ratio',
       'Liquid Assets to Total Assets Ratio',
       'Stable funds to Net Loans Ratio',
       'Liquid Assets to Wholesale funds Ratio',
       'Total Investment Securities to Total Assets Ratio', 'E/A Ratio',
       'D/E Ratio', 'LT Debt/E Ratio', 'Net Profit Margin',
       'Total Assets, Reported', 'Total Deposits', 'Total Debt',
       'Cash and Equivalents', 'Total Equ

In [163]:
# @title Output Test Dataset
data_csv = test.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/Dataset_test_combined.csv', 'w') as f:
  f.write(data_csv)

In [164]:
# @title Handle Outliers

# for x in data.columns[:-2]:
#   if data[x].dtype == object or data[x].dtype == bool: # check if column is non-numeric
#     continue

#   if data[x].unique().shape[0] < 10:
#     print(f"Lesser value: {x}")
#     continue

#   if (x == 'Y' or x == "Year" or x == "Month" ):
#     continue

#   if (x == 'Bond Rating Rank' or x == 'Issuer Rating Rank'):
#     data.loc[data[x] > 13, x] = np.nan
#     continue

#   q75 = np.nanpercentile(data.loc[:, x], 75)
#   q25 = np.nanpercentile(data.loc[:, x], 25)
#   intr_qr = q75 - q25

#   max = q75 + (1.8 * intr_qr)
#   min = q25 - (1.8 * intr_qr)

#   num_outliers = len(data[(data[x] < min) | (data[x] > max)])
#   print(f"{x}: {num_outliers}")

#   data.loc[data[x] < min, x] = np.nan
#   data.loc[data[x] > max, x] = np.nan

# print(data.shape)
# data = data.dropna()
# print(data.shape)

In [165]:
total_num = 0
for name, group in data.groupby("Year"):
  print(name, group['RIC'].unique().shape)

2011.0 (5,)
2012.0 (17,)
2013.0 (29,)
2014.0 (46,)
2015.0 (66,)
2016.0 (101,)
2017.0 (138,)
2018.0 (180,)
2019.0 (232,)
2020.0 (282,)
2021.0 (356,)
2022.0 (446,)


In [166]:
data.info()
data.head(50)
build_EDA_Plots(data)

Output hidden; open in https://colab.research.google.com to view.

In [167]:
# @title Drop columns of Identifier
data = data.drop(['BondID', 'PermID', 'ISIN'], axis = 1)

In [168]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20618 entries, 0 to 22994
Data columns (total 58 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   RIC                                                20618 non-null  object 
 1   IssuePrice                                         20618 non-null  float64
 2   IssueDate                                          20618 non-null  object 
 3   AssetStatusDescription                             20618 non-null  object 
 4   CouponFrequency                                    20618 non-null  float64
 5   CouponRate                                         20618 non-null  float64
 6   DenominationMinimum                                20618 non-null  float64
 7   FaceIssuedTotal                                    20618 non-null  float64
 8   FaceIssuedUSD                                      20618 non-null  int64  
 9   FaceOu

,RIC,IssuePrice,IssueDate,AssetStatusDescription,CouponFrequency,CouponRate,DenominationMinimum,FaceIssuedTotal,FaceIssuedUSD,FaceOutstanding,...,RealGDP,PPI,Unemployment,InterestRate,CoreCPI,Issuer Rating Rank,Bond Rating Rank,Year,Month,Y
0,00135TAA2=,99.771,12/10/2018,Issued,2.0,4.75,200000.0,750000000.0,750000000,136313000.0,...,7.2,3.15,5.3,0.0,0.2,8.333333,10.0,2018.0,11.0,10.0
1,00135TAA2=,99.771,12/10/2018,Issued,2.0,4.75,200000.0,750000000.0,750000000,136313000.0,...,3.5,-2.26,5.3,0.0,0.5,8.333333,10.0,2018.0,12.0,10.0
2,00135TAA2=,99.771,12/10/2018,Issued,2.0,4.75,200000.0,750000000.0,750000000,136313000.0,...,3.5,0.10,5.7,0.0,0.9,8.333333,10.0,2019.0,1.0,10.0
3,00135TAA2=,99.771,12/10/2018,Issued,2.0,4.75,200000.0,750000000.0,750000000,136313000.0,...,3.5,0.10,5.6,0.0,0.7,8.333333,10.0,2019.0,2.0,10.0
4,00135TAA2=,99.771,12/10/2018,Issued,2.0,4.75,200000.0,750000000.0,750000000,136313000.0,...,4.3,0.42,5.4,0.0,1.0,8.333333,10.0,2019.0,3.0,9.0


In [169]:
# @title Normalize and One-hot Encode Data
def transform_data(data):
  for x in data.columns.tolist():
    if (data[x].dtype == object):
      print(f"String: {x}, {data[x].unique().shape}")
      if (x == 'RIC'):
        continue
      if ("Date" in x) or (x == "Maturity"): # Bucket the Date
        Date = data.pop(x)
        Date = pd.to_datetime(Date, format='%d/%m/%Y')
        Date_Bucket = pd.DataFrame({
          f'{x}_year': Date.dt.year,
          f'{x}_month': Date.dt.month,
          f'{x}_day': Date.dt.day,
        })
        for col in Date_Bucket.columns:
          Date_Bucket[col] = (Date_Bucket[col] - Date_Bucket[col].min())/(Date_Bucket[col].max() - Date_Bucket[col].min()) # perform Normalization: Min-Max Scalar
        data = pd.concat([data, Date_Bucket], axis = 1)
      else:
        one_hot_encoded = pd.get_dummies(data[x], prefix = x)
        data = pd.concat([data, one_hot_encoded], axis=1)
        data = data.drop(x, axis=1)
    elif (data[x].dtype == bool):
      print(f"Boolean: {x}")
      data[x] = data[x].astype(int)
    elif (data[x].dtype == int or data[x].dtype == float):
      print(f"Num: {x}")
      if (x == "Y" or x == "Year" or x == "Month" or x == "Prev Bond Rating Rank"): # Skip the y-value
        continue
      if (x == "Bond Rating Rank"):
        data["Prev Bond Rating Rank"] = data[x]

      data[x] = (data[x] - data[x].min())/(data[x].max() - data[x].min()) # perform Normalization: Min-Max Scalar
    else:
      print("null")

  return data

transformed_data = transform_data(data)

String: RIC, (465,)
Num: IssuePrice
String: IssueDate, (318,)
String: AssetStatusDescription, (1,)
Num: CouponFrequency
Num: CouponRate
Num: DenominationMinimum
Num: FaceIssuedTotal
Num: FaceIssuedUSD
Num: FaceOutstanding
String: FaceOutstandingDate, (312,)
Num: FaceOutstandingUSD
String: MoodysRatingDate, (100,)
String: Maturity, (360,)
String: CommonName, (64,)
String: CountryHeadquarters, (15,)
String: Ticker, (63,)
Boolean: IsPublic
Num: BondsCount
Num: Tier 1 Capital Ratio
Num: Total Capital Ratio
Num: Non Performing Loan Ratio
Num: Coverage Ratio
Num: Equity at Risk
Num: Efficiency Ratio
Num: Loan to Deposit Ratio
Num: CASA Ratio
Num: CD Non Equity Ratio
Num: Liquid Assets to Total Assets Ratio
Num: Stable funds to Net Loans Ratio
Num: Liquid Assets to Wholesale funds Ratio
Num: Total Investment Securities to Total Assets Ratio
Num: E/A Ratio
Num: D/E Ratio
Num: LT Debt/E Ratio
Num: Net Profit Margin
Num: Total Assets, Reported
Num: Total Deposits
Num: Total Debt
Num: Cash and Eq

In [170]:
transformed_data.info()
transformed_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20618 entries, 0 to 22994
Columns: 220 entries, RIC to Prev Bond Rating Rank
dtypes: float64(60), int64(1), object(1), uint8(158)
memory usage: 13.0+ MB


,RIC,IssuePrice,CouponFrequency,CouponRate,DenominationMinimum,FaceIssuedTotal,FaceIssuedUSD,FaceOutstanding,FaceOutstandingUSD,IsPublic,...,Country_GBR,Country_IRL,Country_ITA,Country_JPN,Country_KOR,Country_MEX,Country_NLD,Country_TUR,Country_USA,Prev Bond Rating Rank
0,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,...,0,1,0,0,0,0,0,0,0,10.0
1,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,...,0,1,0,0,0,0,0,0,0,10.0
2,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,...,0,1,0,0,0,0,0,0,0,10.0
3,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,...,0,1,0,0,0,0,0,0,0,10.0
4,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,...,0,1,0,0,0,0,0,0,0,10.0


In [171]:
# @title Split Training set and Test Set
grouped = transformed_data.groupby('RIC')

train_1 = pd.DataFrame()

for name, group in grouped:
  train_1 = pd.concat([train_1, group[(group['Year'] < 2022)]], axis = 0)

In [172]:
# @title Output Training Dataset
data_csv = train_1.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/Dataset_train_1.csv', 'w') as f:
  f.write(data_csv)

In [173]:
train_1['Month'] = (train_1['Month'] - train_1['Month'].min())/(train_1['Month'].max() - train_1['Month'].min())
train_1['Year'] = (train_1['Year'] - train_1['Year'].min())/(train_1['Year'].max() - train_1['Year'].min())

train_1 = train_1.reset_index(drop = True)

In [174]:
train_1.info()
train_1.iloc[:, :10].head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15620 entries, 0 to 15619
Columns: 220 entries, RIC to Prev Bond Rating Rank
dtypes: float64(60), int64(1), object(1), uint8(158)
memory usage: 9.7+ MB


,RIC,IssuePrice,CouponFrequency,CouponRate,DenominationMinimum,FaceIssuedTotal,FaceIssuedUSD,FaceOutstanding,FaceOutstandingUSD,IsPublic
0,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
1,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
2,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
3,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
4,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
5,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
6,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
7,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
8,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1
9,00135TAA2=,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1


In [175]:
# @title Address Class Imbalance
from imblearn.over_sampling import SMOTE

# y_train_1 = train_1["Y"]
# X_train_1 = train_1.drop(columns = ["Y", "RIC"])

# smote = SMOTE(sampling_strategy='auto', k_neighbors=1, random_state=42)

# X_train_1_balanced, y_train_1_balanced = smote.fit_resample(X_train_1, y_train_1)

# x_train_balanced = pd.concat([X_train_1, X_train_1_balanced], axis = 0)
# y_train_balanced = pd.concat([y_train_1, y_train_1_balanced], axis = 0)

# balanced_train_1 = pd.concat([x_train_balanced, y_train_balanced], axis = 1)

In [176]:
balanced_train_1 = train_1.drop(columns = ["RIC"])

In [177]:
balanced_train_1.info()
balanced_train_1.head(21)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15620 entries, 0 to 15619
Columns: 219 entries, IssuePrice to Prev Bond Rating Rank
dtypes: float64(60), int64(1), uint8(158)
memory usage: 9.6 MB


,IssuePrice,CouponFrequency,CouponRate,DenominationMinimum,FaceIssuedTotal,FaceIssuedUSD,FaceOutstanding,FaceOutstandingUSD,IsPublic,BondsCount,...,Country_GBR,Country_IRL,Country_ITA,Country_JPN,Country_KOR,Country_MEX,Country_NLD,Country_TUR,Country_USA,Prev Bond Rating Rank
0,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,10.0
1,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,10.0
2,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,10.0
3,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,10.0
4,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,10.0
5,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,9.0
6,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,9.0
7,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,9.0
8,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,9.0
9,0.90545,0.166667,0.559748,0.039808,0.213382,0.213382,0.038677,0.038677,1,0.001136,...,0,1,0,0,0,0,0,0,0,9.0


In [178]:
# build_EDA_Plots(balanced_train_1.iloc[:, 130:])

In [179]:
for name, group in balanced_train_1.groupby('IsPublic'):
  print(group.shape)

print()

(3161, 219)
(12459, 219)



In [180]:
# @title Output Preprocessed Training Dataset
data_csv = balanced_train_1.to_csv(index=False)

with open('/content/drive/My Drive/Colab_Notebooks/TEAM3_TEMG4940C/train_1.csv', 'w') as f:
  f.write(data_csv)